In [1]:
from dictionary_graph import DictionaryGraph
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
def similarity_limit(A, B, iterations=100):
    Z = np.full((B.shape[0], A.shape[0]), 1)
    for i in range(2*iterations): # Paran broj iteracija
        Z = B @ Z @ A.T + B.T @ Z @ A
        Z /= np.linalg.norm(Z, 'fro')
    return Z

In [3]:
graph = DictionaryGraph('OPTED', 'a-z')

def get_synonyms(word, n=10):
    vertices, adjacency_matrix = graph.adjacency_matrix(word)
    S = pd.DataFrame(similarity_limit(
        np.array([
            [0, 1, 0],
            [0, 0, 1],
            [0, 0, 0],
        ], dtype=np.float64),
        adjacency_matrix,
    ), index=vertices)
    return S[1].nlargest(n)

In [4]:
get_synonyms('disappear')

value
disappear    0.372864
vanish       0.196902
pass         0.150385
die          0.123481
fade         0.111587
faint        0.105166
wear         0.091632
gradually    0.087204
light        0.066426
evanid       0.061807
Name: 1, dtype: float64

In [5]:
get_synonyms('parallelogram')

value
parallelogram    0.340849
square           0.231558
rhomb            0.153895
parallel         0.153037
figure           0.152685
angles           0.135860
equal            0.126557
opposite         0.126297
quadrilateral    0.115235
right            0.111697
Name: 1, dtype: float64

In [6]:
get_synonyms('sugar')

value
sugar       0.410236
sucrose     0.068085
cane        0.067927
starch      0.064562
sweet       0.061286
dextrose    0.059788
milk        0.056647
white       0.053995
obtained    0.048164
juice       0.047120
Name: 1, dtype: float64

In [7]:
get_synonyms('science')

value
science        0.410076
art            0.144694
branch         0.078969
study          0.067951
practice       0.051830
knowledge      0.051790
natural        0.050882
learning       0.049035
skill          0.045213
application    0.045135
Name: 1, dtype: float64